In [ ]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["animation.embed_limit"] = 128

import skimage
import skimage.io as sio
import skimage.transform

import yuca
from yuca.ca.neural import NCA
from yuca.ca.continuous import CCA
from yuca.cppn import CPPN

from yuca.zoo.librarian import Librarian
from yuca.kernels import get_kernel

torch.set_default_dtype(torch.float32)

import IPython

from importlib import reload
reload(yuca)
reload(yuca.ca)

In [ ]:
def plot_grid(grid, my_cmap=plt.get_cmap("magma"), title="SmoothLife animation", vmin=0.0, vmax=1):

    global subplot_0
    
    fig, ax = plt.subplots(1,1, figsize=(4.5,4.5), facecolor="white")

    # TODO invert cmap
    
    my_cmap=plt.get_cmap("magma")
    grid_display = 1.0 - my_cmap(np.array(grid[0,0]))[:,:,:3]
    
    subplot_0 = ax.imshow(grid_display, interpolation="nearest")
    
    fig.suptitle(title, fontsize=8)

    ax.set_yticklabels('')
    ax.set_xticklabels('')
    
    plt.tight_layout()

    return fig, ax

def update_fig(i):

    global subplot_0    
    global grid
    #global ax
    
    grid = ca(grid)
    
    my_cmap=plt.get_cmap("magma")
    grid_display = 1.0 - my_cmap(np.array(grid[0,0].cpu()))[:,:,:3]
    
    subplot_0.set_array(grid_display)
        
    plt.tight_layout()
    
def scale_pattern(scale_h, scale_w, pattern):
    
    use_anti_aliasing = True
    
    scaled_pattern = skimage.transform.rescale(pattern, \
            scale=(1,1, scale_h, scale_w), \
            anti_aliasing=use_anti_aliasing,\
            mode="constant", cval=0.0)
    
    return scaled_pattern

In [ ]:
lib = Librarian(verbose=False)

save_figs = False
my_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

lib.index

# SmoothLife

After a warm-up period, 
The SmoothLife glider is stable for ~1024 steps with $K_r = 36$, $\Delta t=0.19$, and 16-bit PyTorch floats,
but unstable by 1024 steps with $K_r = 36$, $\Delta t=0.19$, and 32 or 64-bit PyTorch floats,


In [ ]:
number_frames = 1024
my_radius = 36
warmup_dt = 0.225
my_dt = 0.19
warmup_steps = 192

pattern_name = "smoothlife_single_glider000"
p, m = lib.load(pattern_name)
ca = CCA()

ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)
native_radius = ca.kernel_radius
my_scale = my_radius/native_radius

ca.set_dt(warmup_dt)
ca.set_kernel_radius(my_radius)

p = scale_pattern(my_scale, my_scale, p)

grid_dim = int(96 * my_scale)

grid = ca.initialize_grid(dim=grid_dim).to(my_device)
grid[:,:,32:32+p.shape[-2],32:32+p.shape[-1]] = torch.tensor(p)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("initial grid", fontsize=24)

for my_step in range(warmup_steps):
    grid = ca(grid)
    
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {warmup_steps} warmup steps", fontsize=24)

grid_0 = 1.0 * grid

In [ ]:
if torch.cuda.is_available():
    torch.set_default_dtype(torch.float16)
else:
    torch.set_default_dtype(torch.float32)

pattern_name = "smoothlife_single_glider000"
p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("", fontsize=24)

IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=number_steps, interval=10).to_jshtml())

In [ ]:
torch.set_default_dtype(torch.float32)

pattern_name = "smoothlife_single_glider000"
p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("initial grid", fontsize=24)
IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=number_steps, interval=10).to_jshtml())

In [ ]:
torch.set_default_dtype(torch.float64)

pattern_name = "smoothlife_single_glider000"
p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("", fontsize=24)
IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=number_steps, interval=10).to_jshtml())

# Lenia: _Scutium gravidus_


After a warm-up period of 192 steps, 
The _Scutium gravidus_ glider is stable for ~1024 steps with $K_r = 13$, $\Delta t=0.24$, and 16-bit PyTorch floats,
but unstable with 32 or 64-bit PyTorch floats,


In [ ]:
number_steps = 1024
my_radius = 13
warmup_dt = 0.35
my_dt = 0.24
warmup_steps = 192

pattern_name = "scutium_gravidus_single000"
p, m = lib.load(pattern_name)
ca = CCA()

ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)
native_radius = ca.kernel_radius
my_scale = my_radius/native_radius

ca.set_dt(warmup_dt)
ca.set_kernel_radius(my_radius)

p = scale_pattern(my_scale, my_scale, p)

grid_dim = int(96 * my_scale)

grid = ca.initialize_grid(dim=grid_dim).to(my_device)
grid[:,:,32:32+p.shape[-2],32:32+p.shape[-1]] = torch.tensor(p)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("initial grid", fontsize=24)

for my_step in range(warmup_steps):
    grid = ca(grid)
    
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {warmup_steps} warmup steps", fontsize=24)

grid_0 = 1.0 * grid

In [ ]:
if torch.cuda.is_available():
    torch.set_default_dtype(torch.float16)
else:
    torch.set_default_dtype(torch.float32)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("", fontsize=24)

IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=number_steps, interval=10).to_jshtml())

In [ ]:
torch.set_default_dtype(torch.float32)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("", fontsize=24)

IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=number_steps, interval=10).to_jshtml())

In [ ]:
torch.set_default_dtype(torch.float64)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("", fontsize=24)

IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=number_steps, interval=10).to_jshtml())

# Lenia: _Triscutium solidus_


After a warm-up period of 192 steps, 
The _T. solidus_ glider is unstable for ~1024 steps with $K_r = 11$, $\Delta t=0.09$, and 16-bit PyTorch floats,
but stable with 32 or 64-bit PyTorch floats,


In [ ]:
number_steps = 1024
my_radius = 11
warmup_dt = 0.06
my_dt = 0.09
warmup_steps = 192

if torch.cuda.is_available():
    torch.set_default_dtype(torch.float16)
else:
    torch.set_default_dtype(torch.float32)

pattern_name = "triscutium_solidus_triscutium000"
p, m = lib.load(pattern_name)
ca = CCA()

ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)
native_radius = ca.kernel_radius
my_scale = my_radius/native_radius

ca.set_dt(warmup_dt)
ca.set_kernel_radius(my_radius)

p = scale_pattern(my_scale, my_scale, p)

grid_dim = int(96 * my_scale)

grid = ca.initialize_grid(dim=grid_dim).to(my_device)
grid[:,:,32:32+p.shape[-2],32:32+p.shape[-1]] = torch.tensor(p)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("initial grid", fontsize=24)

for my_step in range(warmup_steps):
    grid = ca(grid)
    
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {warmup_steps} warmup steps", fontsize=24)

grid_0 = 1.0 * grid

In [ ]:
if torch.cuda.is_available():
    torch.set_default_dtype(torch.float16)
else:
    torch.set_default_dtype(torch.float32)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()

#### *********************
torch.set_default_dtype(torch.float32)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()
#### *********************

torch.set_default_dtype(torch.float64)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()
#### *********************

# H. natans wobbler

after 192 steps to warm up, with k_r = 19, dt=0.41,, the _H. natans_ wobbler is stable to about 1024 steps 
for 16 and 32 bit torch floats, but unstable with 64 bits. 

In [ ]:
number_steps = 1024
my_radius = 19
warmup_dt = 0.35
my_dt = 0.41
warmup_steps = 192

if torch.cuda.is_available():
    torch.set_default_dtype(torch.float16)
else:
    torch.set_default_dtype(torch.float32)

pattern_name = "geminium2_wobble_glider000"
p, m = lib.load(pattern_name)
ca = CCA()

ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)
native_radius = ca.kernel_radius
my_scale = my_radius/native_radius

ca.set_dt(warmup_dt)
ca.set_kernel_radius(my_radius)

p = scale_pattern(my_scale, my_scale, p)

grid_dim = int(96 * my_scale)

grid = ca.initialize_grid(dim=grid_dim).to(my_device)
grid[:,:,:p.shape[-2],:p.shape[-1]] = torch.tensor(p)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("initial grid", fontsize=24)

for my_step in range(warmup_steps):
    grid = ca(grid)
    
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {warmup_steps} warmup steps", fontsize=24)

grid_0 = 1.0 * grid

In [ ]:
if torch.cuda.is_available():
    torch.set_default_dtype(torch.float16)
else:
    torch.set_default_dtype(torch.float32)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()

#### *********************
torch.set_default_dtype(torch.float32)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()
#### *********************

torch.set_default_dtype(torch.float64)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()
#### *********************

# s613 wobbler

after 192 steps to warm up, with k_r = 13, dt=0.066,, the s613 wobbler is stable to about 1024 steps 
for 16 bit torch floats, but not with 32, 64 bit floats. 

In [ ]:
number_steps = 1024
my_radius = 13
warmup_dt = 0.08
my_dt = 0.066
warmup_steps = 192

if torch.cuda.is_available():
    torch.set_default_dtype(torch.float16)
else:
    torch.set_default_dtype(torch.float32)

pattern_name = "s613_fast_wobble_glider000"
p, m = lib.load(pattern_name)
ca = CCA()

ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)
native_radius = ca.kernel_radius
my_scale = my_radius/native_radius

ca.set_dt(warmup_dt)
ca.set_kernel_radius(my_radius)

p = scale_pattern(my_scale, my_scale, p)

grid_dim = int(128 * my_scale)

grid = ca.initialize_grid(dim=grid_dim).to(my_device)
grid[:,:,:p.shape[-2],:p.shape[-1]] = torch.tensor(p)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("initial grid", fontsize=24)

for my_step in range(warmup_steps):
    grid = ca(grid)
    
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {warmup_steps} warmup steps", fontsize=24)

grid_0 = 1.0 * grid

In [ ]:
if torch.cuda.is_available():
    torch.set_default_dtype(torch.float16)
else:
    torch.set_default_dtype(torch.float32)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()

#### *********************
torch.set_default_dtype(torch.float32)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()
#### *********************

torch.set_default_dtype(torch.float64)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()
#### *********************

# s643 wobbler

after 192 steps to warm up, with k_r = 21, dt=0.08,, the s643 wobbler is stable to about 1024 steps 
for 16 bit torch floats, but not with 32, 64 bit floats. 

In [ ]:
number_steps = 1024
my_radius = 21
warmup_dt = 0.08
my_dt = 0.08
warmup_steps = 192

if torch.cuda.is_available():
    torch.set_default_dtype(torch.float16)
else:
    torch.set_default_dtype(torch.float32)

pattern_name = "s643_s643_3wide_wobble_glider000"
p, m = lib.load(pattern_name)
ca = CCA()

ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)
native_radius = ca.kernel_radius
my_scale = my_radius/native_radius

ca.set_dt(warmup_dt)
ca.set_kernel_radius(my_radius)

p = scale_pattern(my_scale, my_scale, p)

grid_dim = int(128 * my_scale)

grid = ca.initialize_grid(dim=grid_dim).to(my_device)
grid[:,:,:p.shape[-2],:p.shape[-1]] = torch.tensor(p)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("initial grid", fontsize=24)

for my_step in range(warmup_steps):
    grid = ca(grid)
    
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {warmup_steps} warmup steps", fontsize=24)

grid_0 = 1.0 * grid

In [ ]:
if torch.cuda.is_available():
    torch.set_default_dtype(torch.float16)
else:
    torch.set_default_dtype(torch.float32)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()

#### *********************
torch.set_default_dtype(torch.float32)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()
#### *********************

torch.set_default_dtype(torch.float64)

p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()
#### *********************

# Neurorbium

after 192 steps to warm up, with k_r = 24, dt=0.14,, the neurorbium glider is stable to about 1024 steps 
for 32 bit torch floats, but not with 16 or 64 bit floats. 

In [ ]:
number_steps = 1024
my_radius = 24
warmup_dt = 0.14
my_dt = 0.14
warmup_steps = 192

if torch.cuda.is_available():
    torch.set_default_dtype(torch.float16)
else:
    torch.set_default_dtype(torch.float32)

pattern_name = "neurorbium000"
p, m = lib.load(pattern_name)
ca = NCA()

ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

ca.eval()

native_radius = ca.kernel_radius
my_scale = my_radius/native_radius

ca.set_dt(warmup_dt)
ca.set_kernel_radius(my_radius)

p = scale_pattern(my_scale, my_scale, p)

grid_dim = int(128 * my_scale)

grid = ca.initialize_grid(dim=grid_dim).to(my_device)
grid[:,:,32:32+p.shape[-2],32:32+p.shape[-1]] = torch.tensor(p)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("initial grid", fontsize=24)

for my_step in range(warmup_steps):
    grid = ca(grid)
    
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {warmup_steps} warmup steps", fontsize=24)

grid_0 = 1.0 * grid

In [ ]:
if torch.cuda.is_available():
    torch.set_default_dtype(torch.float16)
else:
    torch.set_default_dtype(torch.float32)

p, m = lib.load(pattern_name)
ca = NCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)
ca.eval()

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()

#### *********************
torch.set_default_dtype(torch.float32)

p, m = lib.load(pattern_name)
ca = NCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)
ca.eval()

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()
#### *********************

torch.set_default_dtype(torch.float64)

p, m = lib.load(pattern_name)
ca = NCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)
ca.eval()

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"initial grid\n {torch.get_default_dtype()}", fontsize=24)

for my_step in range(number_steps):
    grid = ca(grid)
        
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {my_step+1} steps\n {torch.get_default_dtype()}", fontsize=24)
plt.show()
#### *********************